**Output Parsers**
Transform the output of LLM into a more suitable format. 

In [4]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

import os

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))

**CSV Parser**

In [6]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five places {places}. \n {format_instructions}",
        input_variables=["places"],
        partial_variables={"format_instructions": format_instructions},
)

In [7]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"places": "End year countdown to visit on New Year's Eve"})

['Times Square (New York City)',
 'Sydney Harbour (Sydney)',
 'The London Eye (London)',
 'Copacabana Beach (Rio de Janeiro)',
 'Shibuya Crossing (Tokyo)']

**JSON parser**

In [28]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


In [29]:
class Travel(BaseModel):
    places: str = Field(description="List of places to visit on New Year's Eve")
    description: str = Field(description="Description of the places")
    activities: str = Field(description="Activities to do in the places")

In [31]:
travel_query = "List five places to visit on New Year's Eve"

parser = JsonOutputParser(pydantic_object=Travel)

prompt = PromptTemplate(
    template="Answer the user query. \n {format_instructions}\n {query}.\n ",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llm | parser
chain.invoke({"query": travel_query})

{'places': 'New York City, Sydney, London, Las Vegas, Paris',
 'description': "Iconic cities known for their spectacular New Year's Eve celebrations.",
 'activities': 'Fireworks displays, concerts, parties, street celebrations, and fine dining.'}

**Without Pydantic**

In [33]:
travel_query = "List five places to visit on New Year's Eve"
parser = JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query. \n {format_instructions}\n {query}.\n ",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llm | parser
chain.invoke({"query": travel_query})


{'new_years_eve_destinations': [{'place': 'New York City, USA',
   'description': 'Experience the iconic ball drop in Times Square, attend a star-studded concert, or enjoy a dazzling fireworks display over the harbor.'},
  {'place': 'Sydney, Australia',
   'description': 'Witness the spectacular fireworks show over the Sydney Harbour Bridge and Opera House, one of the first major celebrations to ring in the New Year.'},
  {'place': 'London, England',
   'description': 'Enjoy the vibrant atmosphere of London with its impressive fireworks display over the Thames, lively street parties, and festive pub celebrations.'},
  {'place': 'Paris, France',
   'description': "Celebrate in the City of Lights with a romantic dinner cruise on the Seine, a dazzling light show at the Arc de Triomphe, or a festive party in one of the city's many clubs."},
  {'place': 'Reykjavik, Iceland',
   'description': "Experience a unique New Year's Eve with bonfires, fireworks set off by locals, and the chance to s

**Structured Output Parser**

In [34]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser


In [41]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="description", description="detaied description on the answer topic"),
    ResponseSchema(name="applications", description="real world applications of the answer topic"),
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
prompt = PromptTemplate(
    template="Answer the user query as best as possible. \n {format_instructions}\n {question}.\n ",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llm | parser
chain.invoke({"question": "Name a few unexplored invention in fintech in the 21th century will and have yet changed the world."})

{'answer': 'While pinpointing completely "unexplored" inventions is difficult (as many ideas are in development or conceptual stages), here are a few areas in fintech with significant potential for groundbreaking innovation in the 21st century, that are either in nascent stages or have the potential to drastically change the world:\n\n*   **Decentralized Autonomous Organizations (DAOs) for Finance:** DAOs could revolutionize how financial institutions are governed and operated. Imagine a DAO managing a lending protocol, where decisions about interest rates, loan approvals, and risk management are made collectively by token holders through smart contracts. This could lead to more transparent, efficient, and democratic financial systems.\n*   **AI-Powered Hyper-Personalized Financial Advice:** While AI is already used in robo-advisors, the next level involves AI that can deeply understand an individual\'s financial situation, goals, and even psychological biases to provide truly personal